# How to download flights csv file from transtats website

**In this notebook, we will**
1. Download a csv file for your chosen year(s) and month(s)
2. Prepare the data for further processing
3. Push the prepared data to a table in the database




In [1]:
# Import all necessary libraries
import pandas as pd
import numpy as np
import psycopg2
import requests #package for getting data from the web
from zipfile import * #package for unzipping zip files
from sql import get_engine #adjust this as necessary to match your sql.py connection methods

# 1. Download csv file with flight data for your specific year/month

In the following, you are going to download a csv file containing flight data from [this website](https://transtats.bts.gov).    
You can specify, which data you want to download. 

Choose a month/year that you want to explore further.
With the following command lines, you will download a csv file on public flight data from [this website](https://transtats.bts.gov) containing data of your chosen month/year.    
The file will be stored in a data folder.

In [2]:
# Specifies path for saving file
path ='data/' 
# Create the data folder
!mkdir {path}

mkdir: data/: File exists


In [6]:
years = [2012] # list of years you want to look at, specify one year
months = [10, 11] # list of months you want to look at, specify one month
# Here: October 2012

# Loop through months
for year in years:
    for month in months:
        # Get the file from the website https://transtats.bts.gov
        zip_file = f'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_{year}_{month}.zip'
        csv_file = f'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_{year}_{month}.csv'
        url = (f'https://transtats.bts.gov/PREZIP/{zip_file}')
        # Download the database
        r = requests.get(f'{url}', verify=False)
        # Save database to local file storage
        with open(path+zip_file, 'wb') as f:
            f.write(r.content)

/usr/local/Caskroom/miniconda/base/envs/sql-practice/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'transtats.bts.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/usr/local/Caskroom/miniconda/base/envs/sql-practice/lib/python3.8/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'transtats.bts.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [23]:
# Unzip your file
for month in months:
    z_file = f'On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2012_{month}.zip'
    with ZipFile(path+z_file, 'r') as zip_ref:
        zip_ref.extractall(path)

In [3]:
# Read in your data
csv_file = f'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2012_10.csv'
df_10 = pd.read_csv(path+csv_file, low_memory = False)

csv_file = f'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2012_11.csv'
df_11 = pd.read_csv(path+csv_file, low_memory = False)

# df_names = ['df_10', 'df_11']
# for i, month in enumerate(months):
#     csv_file = f'On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2012_{month}.csv'
#     df_names[i] = pd.read_csv(path+csv_file, low_memory = False)

In [39]:
print(df_10.shape)
print(df_11.shape)

(515254, 110)
(488006, 110)


In [7]:
pd.set_option('display.max_columns', None)
display(df_10.head())
df_10[df_10['OriginState'] == 'NJ']

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,DivAirportLandings,DivReachedDest,DivActualElapsedTime,DivArrDelay,DivDistance,Div1Airport,Div1AirportID,Div1AirportSeqID,Div1WheelsOn,Div1TotalGTime,Div1LongestGTime,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
0,2012,4,10,1,1,2012-10-01,AA,19805,AA,N320AA,1,12478,1247802,31703,JFK,"New York, NY",NY,36,New York,22,12892,1289203,32575,LAX,"Los Angeles, CA",CA,6,California,91,900,933.0,33.0,33.0,1.0,2.0,0900-0959,20.0,953.0,1210.0,55.0,1205,1305.0,60.0,60.0,1.0,4.0,1200-1259,0.0,NaN,0.0,365.0,392.0,317.0,1.0,2475.0,10,33.0,0.0,27.0,0.0,0.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012,4,10,2,2,2012-10-02,AA,19805,AA,N319AA,1,12478,1247802,31703,JFK,"New York, NY",NY,36,New York,22,12892,1289203,32575,LAX,"Los Angeles, CA",CA,6,California,91,900,906.0,6.0,6.0,0.0,0.0,0900-0959,16.0,922.0,1150.0,10.0,1205,1200.0,-5.0,0.0,0.0,-1.0,1200-1259,0.0,NaN,0.0,365.0,354.0,328.0,1.0,2475.0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012,4,10,3,3,2012-10-03,AA,19805,AA,N319AA,1,12478,1247802,31703,JFK,"New York, NY",NY,36,New York,22,12892,1289203,32575,LAX,"Los Angeles, CA",CA,6,California,91,900,900.0,0.0,0.0,0.0,0.0,0900-0959,26.0,926.0,1206.0,19.0,1205,1225.0,20.0,20.0,1.0,1.0,1200-1259,0.0,NaN,0.0,365.0,385.0,340.0,1.0,2475.0,10,0.0,0.0,20.0,0.0,0.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012,4,10,4,4,2012-10-04,AA,19805,AA,N325AA,1,12478,1247802,31703,JFK,"New York, NY",NY,36,New York,22,12892,1289203,32575,LAX,"Los Angeles, CA",CA,6,California,91,900,853.0,-7.0,0.0,0.0,-1.0,0900-0959,21.0,914.0,1150.0,15.0,1205,1205.0,0.0,0.0,0.0,0.0,1200-1259,0.0,NaN,0.0,365.0,372.0,336.0,1.0,2475.0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012,4,10,5,5,2012-10-05,AA,19805,AA,N325AA,1,12478,1247802,31703,JFK,"New York, NY",NY,36,New York,22,12892,1289203,32575,LAX,"Los Angeles, CA",CA,6,California,91,900,853.0,-7.0,0.0,0.0,-1.0,0900-0959,28.0,921.0,1229.0,7.0,1205,1236.0,31.0,31.0,1.0,2.0,1200-1259,0.0,NaN,0.0,365.0,403.0,368.0,1.0,2475.0,10,0.0,0.0,31.0,0.0,0.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,DivAirportLandings,DivReachedDest,DivActualElapsedTime,DivArrDelay,DivDistance,Div1Airport,Div1AirportID,Div1AirportSeqID,Div1WheelsOn,Div1TotalGTime,Div1LongestGTime,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
965,2012,4,10,1,1,2012-10-01,AA,19805,AA,N3GVAA,119,11618,1161802,31703,EWR,"Newark, NJ",NJ,34,New Jersey,21,12892,1289203,32575,LAX,"Los Angeles, CA",CA,6,California,91,1835,1827.0,-8.0,0.0,0.0,-1.0,1800-1859,34.0,1901.0,2115.0,14.0,2150,2129.0,-21.0,0.0,0.0,-2.0,2100-2159,0.0,NaN,0.0,375.0,362.0,314.0,1.0,2454.0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
966,2012,4,10,2,2,2012-10-02,AA,19805,AA,N3AWAA,119,11618,1161802,31703,EWR,"Newark, NJ",NJ,34,New Jersey,21,12892,1289203,32575,LAX,"Los Angeles, CA",CA,6,California,91,1835,1920.0,45.0,45.0,1.0,3.0,1800-1859,75.0,2035.0,2254.0,28.0,2150,2322.0,92.0,92.0,1.0,6.0,2100-2159,0.0,NaN,0.0,375.0,422.0,319.0,1.0,2454.0,10,0.0,0.0,55.0,0.0,37.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
967,2012,4,10,3,3,2012-10-03,AA,19805,AA,N3CJAA,119,11618,1161802,31703,EWR,"Newark, NJ",NJ,34,New Jersey,21,12892,1289203,32575,LAX,"Los Angeles, CA",CA,6,California,91,1835,2059.0,144.0,144.0,1.0,9.0,1800-1859,17.0,2116.0,2342.0,5.0,2150,2347.0,117.0,117.0,1.0,7.0,2100-2159,0.0,NaN,0.0,375.0,348.0,326.0,1.0,2454.0,10,0.0,0.0,15.0,0.0,102.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
968,2012,4,10,4,4,2012-10-04,AA,19805,AA,N3GFAA,119,11618,1161802,31703,EWR,"Newark, NJ",NJ,34,New Jersey,21,12892,1289203,32575,LAX,"Los Angeles, CA",CA,6,California,91,1835,1830.0,-5.0,0.0,0.0,-1.0,1800-1859,42.0,1912.0,2142.0,13.0,2150,2155.0,5.0,5.0,0.0,0.0,2100-2159,0.0,NaN,0.0,375.0,385.0,330.0,1.0,2454.0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
969,2012,4,10,5,5,2012-10-05,AA,19805,AA,N3CLAA,119,11618,1161802,31703,EWR,"Newark, NJ",NJ,34,New Jersey,21,12892,1289203,32575,LAX,"Los Angeles, CA",CA,6,California,91,1835,1947.0,72.0,72.0,1.0,4.0,1800-1859,29.0,2016.0,2305.0,6.0,2150,2311.0,81.0,81.0,1.0,5.0,2100-2159,0.0,NaN,0.0,375.0,384.0,349.0,1.0,2454.0,10,16.0,0.0,9.0,0.0,56.0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,N

In [6]:
df_10[df_10['OriginState'] == 'NJ']

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
965,2012,4,10,1,1,2012-10-01,AA,19805,AA,N3GVAA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
966,2012,4,10,2,2,2012-10-02,AA,19805,AA,N3AWAA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
967,2012,4,10,3,3,2012-10-03,AA,19805,AA,N3CJAA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
968,2012,4,10,4,4,2012-10-04,AA,19805,AA,N3GFAA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
969,2012,4,10,5,5,2012-10-05,AA,19805,AA,N3CLAA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510364,2012,4,10,27,6,2012-10-27,MQ,20398,MQ,N515MQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
510365,2012,4,10,28,7,2012-10-28,MQ,20398,MQ,N508MQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
510366,2012,4,10,29,1,2012-10-29,MQ,20398,MQ,N539MQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
510367,2012,4,10,30,2,2012-10-30,MQ,20398,MQ,N521MQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Combine your date
df = df_10.append(df_11)
display(df.shape)
display(df.head())

(1003260, 110)

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
0,2012,4,10,1,1,2012-10-01,AA,19805,AA,N320AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012,4,10,2,2,2012-10-02,AA,19805,AA,N319AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012,4,10,3,3,2012-10-03,AA,19805,AA,N319AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012,4,10,4,4,2012-10-04,AA,19805,AA,N325AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012,4,10,5,5,2012-10-05,AA,19805,AA,N325AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Read in your data
# df = pd.read_csv(path+csv_file, low_memory = False)
# display(df.shape)
# display(df.head())

(515254, 110)

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,...,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Unnamed: 109
0,2012,4,10,1,1,2012-10-01,AA,19805,AA,N320AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012,4,10,2,2,2012-10-02,AA,19805,AA,N319AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012,4,10,3,3,2012-10-03,AA,19805,AA,N319AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012,4,10,4,4,2012-10-04,AA,19805,AA,N325AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012,4,10,5,5,2012-10-05,AA,19805,AA,N325AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1003260 entries, 0 to 488005
Columns: 110 entries, Year to Unnamed: 109
dtypes: float64(70), int64(21), object(19)
memory usage: 849.6+ MB


# 2. Prepare the csv file for further processing

In the next step, we clean and prepare our dataset.

a) Since the dataset consists of a lot of columns, we we define which ones to keep.

In [43]:
# Columns from downloaded file that are to be kept

columns_to_keep = [
                'FlightDate',
                'DepTime',
                'CRSDepTime',
                'DepDelay',
                'ArrTime',
                'CRSArrTime',
                'ArrDelay',
                'Reporting_Airline',
                'Tail_Number',
                'Flight_Number_Reporting_Airline',
                'Origin',
                'Dest',
                'AirTime',
                'Distance',
                'Cancelled',
                'Diverted'
]

In [44]:
df[columns_to_keep].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1003260 entries, 0 to 488005
Data columns (total 16 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   FlightDate                       1003260 non-null  object 
 1   DepTime                          983931 non-null   float64
 2   CRSDepTime                       1003260 non-null  int64  
 3   DepDelay                         983931 non-null   float64
 4   ArrTime                          983215 non-null   float64
 5   CRSArrTime                       1003260 non-null  int64  
 6   ArrDelay                         982034 non-null   float64
 7   Reporting_Airline                1003260 non-null  object 
 8   Tail_Number                      996932 non-null   object 
 9   Flight_Number_Reporting_Airline  1003260 non-null  int64  
 10  Origin                           1003260 non-null  object 
 11  Dest                             1003260 non-null  

In [45]:
# set up your database connection
engine = get_engine()

In [46]:
# The columns in the DB have different naming as in the source csv files. Lets get the names from the DB
table_name_sql = '''SELECT COLUMN_NAME 
                    FROM INFORMATION_SCHEMA.COLUMNS 
                    WHERE TABLE_NAME = 'flights'
                    AND TABLE_SCHEMA ='public'
                    ORDER BY ordinal_position'''
c_names = engine.execute(table_name_sql).fetchall()
c_names

[('flight_date',),
 ('dep_time',),
 ('sched_dep_time',),
 ('dep_delay',),
 ('arr_time',),
 ('sched_arr_time',),
 ('arr_delay',),
 ('airline',),
 ('tail_number',),
 ('flight_number',),
 ('origin',),
 ('dest',),
 ('air_time',),
 ('distance',),
 ('cancelled',),
 ('diverted',)]

In [47]:
# we can clean up the results into a clean list
new_column_names=[]
for name in c_names:
    new_column_names.append(name[0])
new_column_names        

['flight_date',
 'dep_time',
 'sched_dep_time',
 'dep_delay',
 'arr_time',
 'sched_arr_time',
 'arr_delay',
 'airline',
 'tail_number',
 'flight_number',
 'origin',
 'dest',
 'air_time',
 'distance',
 'cancelled',
 'diverted']

In [49]:
# Just in case the above fails here are the results
new_column_names_alternate = ['flight_date',
                              'dep_time',
                              'sched_dep_time',
                              'dep_delay',
                              'arr_time',
                              'sched_arr_time',
                              'arr_delay',
                              'airline',
                              'tail_number',
                              'flight_number',
                              'origin',
                              'dest',
                              'air_time',
                              'distance',
                              'cancelled',
                              'diverted' ]

b) With the next function, we make our csv file ready to be uploaded to SQL.  
We only keep to above specified columns and convert the datatypes.

In [50]:
def clean_airline_df(df):
    '''
    Transforms a df made from BTS csv file into a df that is ready to be uploaded to SQL
    Set rows=0 for no filtering
    '''

    # Build dataframe including only the columns you want to keep
    df_airline = df.loc[:,columns_to_keep]
     
    # Clean data types and NULLs
    df_airline['FlightDate']= pd.to_datetime(df_airline['FlightDate'], yearfirst=True)
    df_airline['CRSArrTime']= pd.to_numeric(df_airline['CRSArrTime'], downcast='integer', errors='coerce')
    df_airline['Cancelled']= pd.to_numeric(df_airline['Cancelled'], downcast='integer')
    df_airline['Diverted']= pd.to_numeric(df_airline['Diverted'], downcast='integer')
    
    # Rename columns
    df_airline.columns = new_column_names
    
    return df_airline

In [51]:
# Call function and check resulting dataframe
df_clean = clean_airline_df(df)
df_clean.head()

,flight_date,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,airline,tail_number,flight_number,origin,dest,air_time,distance,cancelled,diverted
0,2012-10-01,933.0,900,33.0,1305.0,1205,60.0,AA,N320AA,1,JFK,LAX,317.0,2475.0,0,0
1,2012-10-02,906.0,900,6.0,1200.0,1205,-5.0,AA,N319AA,1,JFK,LAX,328.0,2475.0,0,0
2,2012-10-03,900.0,900,0.0,1225.0,1205,20.0,AA,N319AA,1,JFK,LAX,340.0,2475.0,0,0
3,2012-10-04,853.0,900,-7.0,1205.0,1205,0.0,AA,N325AA,1,JFK,LAX,336.0,2475.0,0,0
4,2012-10-05,853.0,900,-7.0,1236.0,1205,31.0,AA,N325AA,1,JFK,LAX,368.0,2475.0,0,0


If you decide to only look at specific airports, it is a good decision to filter for them in advance.  
This function does the filtering. 

In [52]:
# Specify the airports you are interested in and put them as a list in the function.
def select_airport(df, airports):
    ''' Helper function for filtering airline df for a subset of airports'''
    df_out = df.loc[(df.origin.isin(airports)) | (df.dest.isin(airports))]
    return df_out

In [53]:
# Execute function, filtering for New York area airports
airports=['BOS', 'EWR', 'JFK', 'MIA', 'PHI', 'SJU']
if len(airports) > 0:
    df_selected_airports = select_airport(df_clean, airports)
else:
    df_selected_airports = df_clean
    
df_selected_airports.head()

,flight_date,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,airline,tail_number,flight_number,origin,dest,air_time,distance,cancelled,diverted
0,2012-10-01,933.0,900,33.0,1305.0,1205,60.0,AA,N320AA,1,JFK,LAX,317.0,2475.0,0,0
1,2012-10-02,906.0,900,6.0,1200.0,1205,-5.0,AA,N319AA,1,JFK,LAX,328.0,2475.0,0,0
2,2012-10-03,900.0,900,0.0,1225.0,1205,20.0,AA,N319AA,1,JFK,LAX,340.0,2475.0,0,0
3,2012-10-04,853.0,900,-7.0,1205.0,1205,0.0,AA,N325AA,1,JFK,LAX,336.0,2475.0,0,0
4,2012-10-05,853.0,900,-7.0,1236.0,1205,31.0,AA,N325AA,1,JFK,LAX,368.0,2475.0,0,0


# 3. Push the prepared data to a table in the database

In [55]:
# Specify which table within your database you want to push your data to. Give your table an unambiguous name.
# Example: flights_sp for Sina's flights table
table_name = 'flight_api_proj_gr4_raw'
# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This will take a minute or two...

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        df_selected_airports.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None

The flight_api_proj_gr4_raw table was imported successfully.


In [56]:
# Check the number of rows match
table_name_sql = f'''SELECT count(*) 
                    FROM {table_name}
                    '''
engine.execute(table_name_sql).fetchall()[0][0] == df_selected_airports.shape[0]

True

# 4. EDA on the flights data